In [1]:
"""
本地是3.8的python兼容不了1.X版本的tf，所以改了一下提供的代码，完成的任务还是大致一致的
"""
import tensorflow as tf
import numpy as np
tf.executing_eagerly()

learning_rate = 1e-4
keep_prob_rate = 0.7 
max_epoch = 2000
batch_size = 100

mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(-1, 28, 28, 1) / 255.0
x_test = x_test.reshape(-1, 28, 28, 1) / 255.0
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (7, 7), activation='relu', input_shape=(28, 28, 1), padding='same'),
    tf.keras.layers.MaxPooling2D((2, 2), strides=2, padding='same'),
    tf.keras.layers.Conv2D(64, (5, 5), activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D((2, 2), strides=2, padding='same'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dropout(1 - keep_prob_rate),
    tf.keras.layers.Dense(10, activation='softmax')
])

optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
loss_fn = tf.keras.losses.CategoricalCrossentropy()

def train_step(x, y):
    with tf.GradientTape() as tape:
        predictions = model(x, training=True)
        loss = loss_fn(y, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

def compute_accuracy(preds, labels):
    return tf.reduce_mean(tf.cast(tf.equal(tf.argmax(preds, axis=1), 
                                          tf.argmax(labels, axis=1)), 
                                tf.float32))

for epoch in range(max_epoch):
    indices = np.random.permutation(len(x_train))[:batch_size]
    batch_x = x_train[indices]
    batch_y = y_train[indices]
    
    loss = train_step(batch_x, batch_y)
    
    if epoch % 100 == 0:
        predictions = model(x_test[:1000], training=False)
        accuracy = compute_accuracy(predictions, y_test[:1000])
        print(f'Iteration {epoch}, Test Accuracy: {accuracy.numpy():.4f}')

predictions = model(x_test, training=False)
final_accuracy = compute_accuracy(predictions, y_test)
print(f'Final Test Accuracy: {final_accuracy.numpy():.4f}')

Iteration 0, Test Accuracy: 0.1720
Iteration 100, Test Accuracy: 0.8720
Iteration 200, Test Accuracy: 0.9130
Iteration 300, Test Accuracy: 0.9290
Iteration 400, Test Accuracy: 0.9510
Iteration 500, Test Accuracy: 0.9560
Iteration 600, Test Accuracy: 0.9660
Iteration 700, Test Accuracy: 0.9680
Iteration 800, Test Accuracy: 0.9740
Iteration 900, Test Accuracy: 0.9740
Iteration 1000, Test Accuracy: 0.9820
Iteration 1100, Test Accuracy: 0.9800
Iteration 1200, Test Accuracy: 0.9800
Iteration 1300, Test Accuracy: 0.9820
Iteration 1400, Test Accuracy: 0.9820
Iteration 1500, Test Accuracy: 0.9810
Iteration 1600, Test Accuracy: 0.9790
Iteration 1700, Test Accuracy: 0.9850
Iteration 1800, Test Accuracy: 0.9860
Iteration 1900, Test Accuracy: 0.9840
Final Test Accuracy: 0.9840
